In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import gdown
import os
from PIL import Image
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [3]:
torch.manual_seed(3407)

# Create a data transformation pipeline
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [4]:
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = os.path.join(self.root_dir, self.image_paths[idx])
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
      
        return image

In [5]:
test = ImageDataset(root_dir='/content/drive/MyDrive/Test', transform=transform)
N = len(test)
print(f'The number of images in the test set = {N}')

batch_size = 64
testloader = DataLoader(test, batch_size=batch_size, shuffle=False, num_workers=2)

The number of images in the test set = 12


In [6]:
# import matplotlib.pyplot as plt
# for images in testloader:
#   np_images = images.numpy()
#   np_images = np.transpose(np_images, (0, 2, 3, 1))
#   for image in np_images:
#       plt.imshow(image)
#       plt.axis('off')
#       plt.show()

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Availabe device for training is: {device}')

Availabe device for training is: cpu


In [8]:
def predict(model, dataloader, device):
    y_pred = []
    with torch.no_grad():
        for data in dataloader:
            inputs = data
            inputs = inputs.to(device)
            outputs = model(inputs)

            # Return the value and the index of the maximum number of the output vector
            predicted = torch.max(outputs, 1)

            # Save the predicted values from the model of the current batch
            predicted = predicted.indices.cpu().numpy()
            y_pred.extend(predicted)
            
    return np.asarray(y_pred)


In [9]:
# Load model
gdown.download("https://drive.google.com/file/d/1nb4Af9x9MvtsEZVU2lybGmV8Nxt1LPbh/view?usp=sharing", "resnet50.pth", quiet=False, fuzzy=True)
# gdown.download("https://drive.google.com/file/d/1lC80T2HcTkxc7j5mHgRkrcTK93jaaPsP/view?usp=share_link", "alexnet.pth", quiet=False, fuzzy=True)
# gdown.download("https://drive.google.com/file/d/1aD3KmnIkjbBrwJ8BtyYhLuD_bbdr92E-/view?usp=share_link", "densenet201.pth", quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1nb4Af9x9MvtsEZVU2lybGmV8Nxt1LPbh
To: /content/resnet50.pth
100%|██████████| 94.4M/94.4M [00:00<00:00, 94.6MB/s]


'resnet50.pth'

In [10]:
resnet_model = torch.load('./resnet50.pth', map_location=device)
# alexnet_model = torch.load('./alexnet.pth', map_location=device)
# densenet_model = torch.load('./densenet201.pth', map_location=device)

In [11]:
# Predict the test set
y_resnet = predict(resnet_model, testloader, device)
# y_alexnet = predict(alexnet_model, testloader, device)
# y_densnet = predict(densenet_model, testloader, device)

In [12]:
final_predictions = np.asarray([1 - x for x in y_resnet])
np.savetxt('16.txt', final_predictions, delimiter='\n', newline='\n', fmt='%d')
print(final_predictions)

[0 0 0 0 0 0 0 0 0 0 0 0]
